In [1]:
import pandas as pd
import os
import sys
from tqdm import tqdm
from IPython.display import clear_output    


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yash07yadav/project-data")

print("Path to dataset files:", path)

Path to dataset files: /home/ml/.cache/kagglehub/datasets/yash07yadav/project-data/versions/1


In [3]:
path = "/home/ml/.cache/kagglehub/datasets/yash07yadav/project-data/versions/1"

In [4]:
new_path = os.path.join(path, "Complete Dataset")

In [5]:
train_data_path = os.path.join(path, "Complete Dataset", "train")
val_data_path = os.path.join(path, "Complete Dataset", "val")


hockeyFight_train_data_path = os.path.join(train_data_path, "HockeyFight")
movieFights_train_data_path = os.path.join(train_data_path, "MovieFights")
fight_train_data_path = os.path.join(train_data_path, "Fight")
nonFight_train_data_path = os.path.join(train_data_path, "NonFight")


hockeyFight_val_data_path = os.path.join(val_data_path, "HockeyFight")
movieFights_val_data_path = os.path.join(val_data_path, "MovieFights")
fight_val_data_path = os.path.join(val_data_path, "Fight")
nonFight_val_data_path = os.path.join(val_data_path, "NonFight")




In [6]:
print(len(os.listdir(hockeyFight_train_data_path)))
print(len(os.listdir(movieFights_train_data_path)))
print(len(os.listdir(fight_train_data_path)))
print(len(os.listdir(nonFight_train_data_path)))

print("-"*30)

print(len(os.listdir(hockeyFight_val_data_path)))
print(len(os.listdir(movieFights_val_data_path)))
print(len(os.listdir(fight_val_data_path)))
print(len(os.listdir(nonFight_val_data_path)))

400
188
1000
1000
------------------------------
100
35
275
275


In [7]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [8]:
from src.config import DATA_DIR

In [9]:
from src.pipelines.video_preprocessor import VideoDataExtractor
processor = VideoDataExtractor()

Using GPU: NVIDIA GeForce RTX 3050 Laptop GPU
Detection model on: cuda:0
Pose model on: cuda:0


In [10]:
from IPython.display import clear_output
from tqdm.notebook import tqdm


In [14]:
def extract_data(directory_path, csv_path):
    all_files = os.listdir(directory_path)
    for i in tqdm(range(len(all_files))):
        frame_width, frame_height, num_interactions = processor.extract_video_data(
        os.path.join(directory_path, all_files[i]), csv_path
    )
    

In [15]:
extract_data(hockeyFight_train_data_path, os.path.join(DATA_DIR , "hockeyfight_train.csv")) 


  0%|          | 0/400 [00:00<?, ?it/s]

Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Saved 165 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/hockeyfight_train.csv
Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Appended 85 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/hockeyfight_train.csv
Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Appended 9 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/hockeyfight_train.csv
Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Appended 146 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/hockeyfight_train.csv
Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Appended 16 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/hockeyfight_train.csv
Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Appended 287 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/hockeyfight_train.csv
Processing 

In [ ]:
extract_data(movieFights_train_data_path, os.path.join(DATA_DIR , "moviefight_train.csv")) 

  0%|          | 0/188 [00:00<?, ?it/s]

Processing video: 1920x800 at 23.98 fps
Using frame_skip: 1
Saved 3 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/moviefight_train.csv
Processing video: 1920x800 at 23.976 fps
Using frame_skip: 1
Appended 52 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/moviefight_train.csv
Processing video: 720x576 at 25.0 fps
Using frame_skip: 1
Appended 42 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/moviefight_train.csv
Processing video: 1920x816 at 23.98 fps
Using frame_skip: 1
Appended 311 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/moviefight_train.csv
Processing video: 1920x800 at 23.976 fps
Using frame_skip: 1
Appended 47 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/moviefight_train.csv
Processing video: 720x480 at 29.97002997002997 fps
Using frame_skip: 1
Appended 32 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/moviefight_train.

In [ ]:
extract_data(fight_train_data_path, os.path.join(DATA_DIR , "fight_train.csv")) 


In [ ]:
extract_data(nonFight_train_data_path, os.path.join(DATA_DIR , "non-fight_train.csv")) 


validation dataset

In [ ]:
extract_data(hockeyFight_val_data_path, os.path.join(DATA_DIR , "hockeyfight_val.csv")) 


In [ ]:
extract_data(movieFights_val_data_path, os.path.join(DATA_DIR , "moviefight_val.csv")) 


In [ ]:
extract_data(fight_val_data_path, os.path.join(DATA_DIR , "fight_val.csv")) 

In [ ]:
extract_data(nonFight_val_data_path, os.path.join(DATA_DIR , "non-fight_val.csv")) 